In [1]:
# Own Packages
from Masterarbeit_utils.model_utils_seq_class import load_and_modify_model, get_tokenizer

# Site-Packages
import dask.dataframe as dd
import torch
import psutil
import os
import sys
import pickle as pk
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import tensorflow as tf
%load_ext tensorboard


from transformers import AutoTokenizer, AutoConfig, OPTForSequenceClassification
from tokenizers.processors import TemplateProcessing
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset
sys.version, sys.executable

2023-08-14 14:47:49.188331: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-14 14:47:49.206289: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 14:47:49.624083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


('3.10.0 (default, Jul 12 2023, 08:49:30) [GCC 12.2.0]',
 '/home/worker/.pyenv/versions/3.10.0/bin/python')

In [2]:
choices = ['calculate all', 'ask for userinput', 'just calculate needed']
calculation_profile =  choices[2]
calculation_profile

'just calculate needed'

# Parameters

In [3]:
"""
The Paths to important folders have to be changed for your system.
"""

# Name of this experiment
model_name = 'gal_125_seq_2'

# This folder will be created and filled with txt.files for each sample after you run the Pytorch Dataset Notebook
dataset_folder = f'data/dataset_samples'

# The folder at which the model will be saved. This folder has to be created for your system 
model_folder = f'data/models/{model_name}'
os.makedirs(model_folder, exist_ok=True)


# Folder in which the tokenizer will be saved
tokenizer_folder = f'data/tokenizers/{model_name}'
os.makedirs(tokenizer_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
# if default_type = float16 fp16 must be False
default_dtype = torch.bfloat16
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cuda:0'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False
n_f_terms = None # Will be calculated

# Training parameters!
output_dir = model_folder
num_train_epochs = 3
per_device_train_batch_size = 15
per_device_eval_batch_size = 60
gradient_accumulation_steps = 10
save_strategy = "epoch"
logging_strategy = "steps"
evaluation_strategy = "steps"
logging_steps = 10
evaluation_steps = 10000
save_steps = 8000
logging_first_step = True
logging_nan_inf_filter = False


learning_rate = 2e-4 
weight_decay = 0.0  # Parameter from first model run
seed = 42
resume_from_checkpoint = False

# This that could improve performance
dataloader_num_workers = 8
# sytem varables that must be set for the tokenizer
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
torch_compile = False
# V-Ram reduction only if default_dtype= float32
fp16=False
if default_dtype == torch.float16:
    fp16=False
bf16=False
tf32=True

# Creating the Tokenizer

In [4]:
if calculation_profile == choices[0]:
    i = 'y'
elif calculation_profile == choices[1]:  
    i = input("This creates a new tokenizer instance and saves it, if you want to proceed write y: ")
else:
    i = 'n'

if i != 'y' and os.path.isfile(f'{tokenizer_folder}/tokenizer.json'):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_folder)
    n_f_terms = len(tokenizer) - tokenizer.vocab_size
    print('Loadede Tokenizer from serialized instance!')    
    print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')
    tokenizer.padding_side = 'left'
    
else:
    print('generating new tokenizer')
    # Loads a pretrained Tokenizer for the galactica model and adds an additional token for each F-Term
    tokenizer = get_tokenizer(dump_dir)
    
    # The Tokenizer contained initially 50000 Tokens which are stored as the vocab-size.
    # The vocab_size attribute is not updated when the additional tokens are added to the tokenizer
    n_f_terms = len(tokenizer) - tokenizer.vocab_size
    tokenizer.save_pretrained(tokenizer_folder)
    print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')


#!!!! Important
tokenizer.padding_side = 'left'

Loadede Tokenizer from serialized instance!
There are 378166 different F-Terms in the whole Dataset!


# Creating the dataset

In [5]:
class JapPatDataset(Dataset):
    """Dataset containing Japanese patents and their F-Term classification.
    This variant is adapted for sequence classification and returns the f_terms as a list of labels"""
    def __init__(self, data_folder, tokenizer):
        """
        data_folder: path to folder containing the text samples
        tokenizer: tokenizer instance with added additional Tokens for F-Terms
        """
        super(Dataset).__init__()
        self.data_folder = data_folder
        # This has to be manually set to the ammount of files in the 'dataset_samples' folder. Calculating the number of files in this folder would take forever.
        # A to low number would lead to samples missing from the dataset.
        # A to high number would raise a FileNotFound error.
        self.l = len(os.listdir(data_folder))
        self.start_f_term_token = '<START F-TERMS>'
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        try:
            with open(f'{self.data_folder}/{idx}.txt', 'r', encoding='utf-8') as f:
                item = f.read()
        except FileNotFoundError:
            raise FileNotFoundError

        #tokenizing the whole sample which will be later split into tokens and labels
        tokenized = self.tokenizer(item)
        tokenized.pop('token_type_ids')
        attention_mask = tokenized.pop('attention_mask')
        tokens = tokenized.pop('input_ids')
        
        tokens = torch.tensor(tokens)
        # separating the abstract text tokens from the f_terms
        input_ids = tokens[tokens < 50000].tolist()
        f_term_ids = tokens[tokens >= 50002] - 50000
        # rescaling the attention_mask to the shorter sequence
        attention_mask = attention_mask[:len(input_ids)]

        # creating a multi hot vector as the label 
        n_f_terms = len(self.tokenizer) - self.tokenizer.vocab_size
        labels = torch.zeros([n_f_terms])
        labels[f_term_ids] = 1
        return {'input_ids': input_ids, 'attention_mask': attention_mask,   'labels':labels.tolist()}

In [6]:
train_dataset = JapPatDataset(f'{dataset_folder}/train', tokenizer)
validation_dataset = JapPatDataset(f'{dataset_folder}/validation', tokenizer)

##### Debugging remove later
#validation_dataset.l = 10000

In [7]:
# The pretrained model is loaded from Huggingface.
# The token-embedding is expanded for all f-terms and the output embeddings is compleatly replaced by a F-Term classification head.
model = load_and_modify_model(base_model_name, default_dtype, tensor_parallel, num_gpus, n_f_terms, default_device)
print(f'The model interprets token-index {model.config.bos_token_id} as the beginning of a sequence and {model.config.eos_token_id} as the end')

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/galactica-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model interprets token-index 0 as the beginning of a sequence and 2 as the end


In [8]:
# Input Text
text = 'Good morning Mr'
# Convert text to tokens
tokens  = tokenizer(text, return_tensors='pt').input_ids
print(f'Output of Tokenizer: {tokens}')

# creating one forward pass
tokens = tokens.to(default_device)
tokens = tokens[:,:-1]

labels = torch.zeros([2, 378166])
labels[:, 100] = 1
labels[:, 1233] = 1
print(labels.view(-1).shape, tokens.shape)
model_output = model(tokens)

Output of Tokenizer: tensor([[    0, 34848, 16810, 14782,     2]])
torch.Size([756332]) torch.Size([1, 4])


In [9]:
f'The model has {model_output["logits"].shape[-1]} output-features, the tokenizer has {len(tokenizer)} tokens'

'The model has 378166 output-features, the tokenizer has 428166 tokens'

# Creating the Trainer Class by Subclassing from Huggingface-Trainer

In [10]:
"""
Subclassing the Huggingface Trainer class to use custome code to calculate the loss
The labels used for the loss are generated and the labels for the text tokens are set to -100 to ignore their loss,
because the modified model can't predict text-tokens
Also changing the log method to save the logs in a tensorboard format.
"""


def generate_log_function():
    """
    This function returns a logging-function that can be used as a method for the CustomTrainer class

    :log_dir:  path to folder in which the logs will be saved
    """
    writer = torch.utils.tensorboard.SummaryWriter()

    def log(self, logs) -> None:
        """
        Log `logs` on the various objects watching training.

        Subclass and override this method to inject custom behavior.

        Args:
            logs (`Dict[str, float]`):
                The values to log.
        """
        # logging is printed after each - logging step but no update on the screen
        if self.state.epoch is not None:
            logs["epoch"] = round(self.state.epoch, 2)

        output = {**logs, **{"step": self.state.global_step}}
        self.state.log_history.append(output)
        self.control = self.callback_handler.on_log(self.args, self.state, self.control, logs)
        for key, value in output.items():
            writer.add_scalar(key, value)
        writer.flush()
    return log


log_function = generate_log_function()
# Just beeing save and checking the right padding position of the tokenizer
tokenizer.padding_side = 'left'


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs: bool=False, loss_fc=torch.nn.CrossEntropyLoss()):
        """
        model: model which should be trained.
        inputs: A padded batch of samples from the dataset.
        return_outputs: Indicates if the whole output of the model is returned or not.
        loss_fc: Instance of a loss function which should be used for the loss calculation
        """
        def cross_entropy_loss(logits, target):
            """
            This function applies softmax and than cross entropy loss to a logits, target pair
        
            A custom CrossEntropy-Loss function that can be applied to multi Label Problems
            """
            # Durch 1.0 Teilen erhält den Gradienten
            predicted_distribution = torch.nn.functional.softmax(logits, -1)/1.0
            # Scaling the predicted distribution to match the target distribution
                    
            n_targ = target.sum(-1).unsqueeze(-1)
            predicted_distribution *= n_targ
                    
            epsilon = 1e-10
            cross_entropy = target * torch.log(predicted_distribution + epsilon)
            cross_entropy = cross_entropy.sum(-1)
            return -torch.mean(cross_entropy)
            
        # Removing the token_type_ids because we don't need them
        try:
            inputs.pop('token_type_ids')
        except KeyError:
            pass
        # extracting and normalizing the labels
        labels = inputs.pop('labels')
        labels = torch.nn.functional.normalize(labels, p=1 , dim = -1)
        # Forward pass
        outputs = model(**inputs, output_attentions=False, output_hidden_states=False, return_dict=True)
        logits = outputs['logits']

        # calculating the loss
        # Stabilize the logarithm calculation by adding a small constant
        
        loss = cross_entropy_loss(logits, labels)
        
        message = f'loss: {loss.item()}'
        sys.stdout.write('\r'+ message)
        
        return (loss, outputs) if return_outputs else loss

    def prediction_step(
        self,
        model: torch.nn.Module,
        inputs: dict,
        prediction_loss_only: bool,
        ignore_keys: list = None,
        ) -> tuple:
        #torch.cuda.empty_cache()
        model = model.eval()
        with torch.no_grad():
            with self.compute_loss_context_manager():
                loss, outputs = self.compute_loss(model, inputs, return_outputs=True)

        return loss, None, None

    def log(self, logs) -> None:
        """
        Log `logs` on the various objects watching training.

        Subclass and override this method to inject custom behavior.

        Args:
            logs (`Dict[str, float]`):
                The values to log.
        """
        log_function(self, logs)


# Training the Model

In [11]:
# The TrainingArguments class is a class which stores multiple parameters for the Custom-trainer of the model.

training_args = TrainingArguments(
    output_dir=output_dir,              
    num_train_epochs=num_train_epochs,             
    per_device_train_batch_size=per_device_train_batch_size,    # batch size per device during training
    per_device_eval_batch_size=per_device_eval_batch_size,
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,
    eval_steps=evaluation_steps,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_first_step=logging_first_step,
    logging_steps=logging_steps,
    save_steps=save_steps,
    logging_nan_inf_filter=logging_nan_inf_filter,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    seed=seed,
    dataloader_num_workers=dataloader_num_workers, 
    fp16=fp16,
    bf16=bf16,
    tf32=tf32,
    torch_compile=torch_compile
    #,
    #adam_beta1=adam_beta1,
    #adam_beta2=adam_beta2,
    #warmup_steps=warmup_steps

)
# Allow the training of the input embeddings
model.enable_input_require_grads()
trainer = CustomTrainer(model=model,
                        args=training_args, 
                        train_dataset=train_dataset, 
                        eval_dataset=validation_dataset,
                        data_collator=DataCollatorWithPadding(tokenizer,
                                                              return_tensors='pt'))

trainer.save_model(f'{output_dir}/checkpoint-0')
train_results = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

/home/worker/.pyenv/versions/3.10.0/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` met

loss: 12.9375

Step,Training Loss,Validation Loss


loss: 11.7555

Exception in thread Thread-7 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/worker/.pyenv/versions/3.10.0/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/worker/.pyenv/versions/3.10.0/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/home/worker/.pyenv/versions/3.10.0/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 51, in _pin_memory_loop
    do_one_step()
  File "/home/worker/.pyenv/versions/3.10.0/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/worker/.pyenv/versions/3.10.0/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/home/worker/.pyenv/versions/3.10.0/lib/python3.10/site-packages/torch/multiprocessing/reductions.py", line 307, in rebuild_storage_fd
    fd = df.detach()
 

KeyboardInterrupt: 

In [19]:
def cross_entropy_loss(logits, target):
            """
            This function applies softmax and than cross entropy loss to a logits, target pair
        
            A custom CrossEntropy-Loss function that can be applied to multi Label Problems
            """
            # Durch 1.0 Teilen erhält den Gradienten
            predicted_distribution = torch.nn.functional.softmax(logits, -1)/1.0
            # Scaling the predicted distribution to match the target distribution
                    
            n_targ = target.sum(-1).unsqueeze(-1)
            predicted_distribution *= n_targ
                    
            epsilon = 1e-10
            cross_entropy = target * torch.log(predicted_distribution + epsilon)
            cross_entropy = cross_entropy.sum(-1)
            return -torch.mean(cross_entropy)


l = torch.tensor([[000, 0, 000, 1]])/1.0
t = torch.tensor([[0, 1, 0, 1]])/1.0

cross_entropy_loss(l, t)

tensor(1.1094)